# Play with Mapping
This notebook lets us play with the mapping that we can create guide the MappedHD5Ingestor class into producing a document stream (ingesting) from an hdf5 file.

1. Build a projection
A [projection](https://blueskyproject.io/event-model/data-model.html#projections) maps from metadata and fields in a docstream to a datastructure keyed on keys with a known ontology. Ontologies could be a structure like fields needed to recreate a NeXus file, or fields needed to display data in an application, like Splash.

2. Import projections file
For now, we have a projection stored in [projections.py](./projections.py). We're using python instead of json because we could add comments. But as long as you use double quotes instead of single quotes for strings and keys, the format is almost the same. This projection will be added to the start doc when we ingest the file.

3. Build a file
First, let's create a sample hdf5 file. We can modify the fields here and watch them update down below when we generate the docstream.

4. Build a mapping
Let's work on the mapping. You create a file...could be python or json, and provide it to the `ingestor`. The exact mechnanism for doing that has not quite been designed, yet. 

For now, we have a mapping stored in [mapping.py](./mapping.py). We're using python instead of json because we could add comments. But as long as you use double quotes instead of single quotes for strings and keys, the format is almost the same.

5. Import the mapping

In [4]:
import datetime
from importlib import reload
import json
import os
import pytz
import sys
sys.path.append("../..") 
import tempfile
from IPython.utils.tempdir import TemporaryWorkingDirectory
from IPython.display import FileLink
import h5py
import numpy as np
from splash_ingest import MappedHD5Ingestor, MappingNotFoundError
from splash_ingest.model import Mapping

def build_file():
    temp_dir = TemporaryWorkingDirectory()
    local = pytz.timezone("America/Los_Angeles")
    num_frames = 3
    date_data_type = h5py.string_dtype(encoding='ascii')
    image_date = np.empty((num_frames), dtype=object)
    scan_date = np.empty((num_frames), dtype=object)
    data = np.empty((num_frames, 16, 9))
    data_dark = np.empty((1, 16, 9))
    data_white = np.empty((1, 16, 9))

    # three data frames
    for x in range(0, num_frames):
        image_date[x] = (str(local.localize(datetime.datetime.now(), is_dst=None)))
        scan_date[x] = (str(local.localize(datetime.datetime.now(), is_dst=None)))
        data[x, :, :] = np.random.random_sample((16, 9))
    # one dark, one white
    data_dark[0, :, :] = np.random.random_sample((16, 9))
    data_white[0, :, :] = np.random.random_sample((16, 9))


    image_date[x] = (str(local.localize(datetime.datetime.now(), is_dst=None)))
    file = h5py.File(os.path.join(temp_dir.name, 'test.hdf5'), 'w')
    file.create_dataset('/measurement/sample/name', data='nifty sample')
    file.create_dataset('/measurement/instrument/name', data='my station')
    file.create_dataset('/measurement/instrument/source/beamline', data='my beam')
#     file.create_dataset('/measurement/instrument/source', data='als')
    file.create_dataset('/process/acquisition/image_date', data=image_date, dtype=date_data_type)
    file.create_dataset('/process/acquisition/scan_date', data=scan_date, dtype=date_data_type)
    file.create_dataset('/exchange/data', data=data)
    file.create_dataset('/exchange/dark', data=data_dark)
    file.create_dataset('/process/acquisition/sample_position_x', data=[1.0, 2.0])
    file.close()
    # reopen for reading
    file = h5py.File(os.path.join(temp_dir.name, 'test.hdf5'), 'r')
    yield file
    file.close()

def build_projections():
    projections_dict = {}
    # with open('mapping.json') as json_file:
    #    mapping_dict = json.load(json_file)
    import projections

    reload(projections)
    return projections.projections

def build_mapping():
    mapping_dict = {}
    # with open('mapping.json') as json_file:
    #    mapping_dict = json.load(json_file)
    import mapping
    reload(mapping)
    mapping_dict = mapping.mapping_dict
    # construct a mapping object from dict to validate that we typed it correctly
    return Mapping(**mapping_dict)

## Ingest!
Now we construct an instance of the MappedHD5Ingestor and ask it to generate us a docstream that reads the mapping and provides fields from the file

The root directory 'test_root' variable that can help us find the file based on a configurable root dir. It will be written directly into the resource document.


In [5]:
detailed_output = True

file_gen = build_file()

file = next(build_file())
mapping = build_mapping()
projections = build_projections()

ingestor = MappedHD5Ingestor(mapping, file, 'find_me_in_the_resource_document', projections=projections)

start_doc = {}
stop_doc = {}

# fill up a dictionary to later run a projection from
from databroker.core import BlueskyRun, SingleRunCache
run_cache = SingleRunCache()
try:
    for name, doc in ingestor.generate_docstream():
        run_cache.callback(name, doc)
        if name == "start":
            start_doc = doc
        if detailed_output:
            print("\n\n===============")
            print("Document:  " + name)
            if name == 'event':
                print(repr(doc))
            else:
                print (json.dumps(doc, indent=1))
        else:
            if name == 'start' or name == 'stop':
                doc_str = json.dumps(doc, indent=1)
                print (f"{name}: {doc_str}")
            else:
                print(name)
       
            
except MappingNotFoundError as e:
    print('Indigestion! ' + repr(e))
    
next(file_gen) # cleanup
run = run_cache.retrieve()




Document:  start
{
 "uid": "a6f9a738-902a-4b50-ad6d-97df51141a95",
 "time": 1602265758.5554955,
 ":measurement:sample:name": "nifty sample",
 ":measurement:instrument:name": "my station",
 "projections": [
  {
   "name": "app_display",
   "version": "42.0.0",
   "configuration": {},
   "projection": {
    "data": {
     "type": "linked",
     "location": "event",
     "stream": "primary",
     "field": ":exchange:data"
    },
    "darks": {
     "type": "linked",
     "location": "event",
     "stream": "darks",
     "field": ":exchange:dark"
    },
    "sample_name": {
     "type": "configuration",
     "field": ":measurement:sample:name",
     "location": "start"
    },
    "instrument_name": {
     "type": "configuration",
     "field": ":measurement:instrument:name",
     "location": "start"
    }
   }
  }
 ]
}


Document:  resource
{
 "uid": "e84795f0-3b63-4c95-bf8f-62c9d4ff992d",
 "run_start": "a6f9a738-902a-4b50-ad6d-97df51141a95",
 "spec": "MultiKeySlice",
 "root": "find_me_i

/home/dylan/work/bluesky/databroker/databroker/core.py:1920: UserWarning: There are 3 entrypoints for the databroker handler spec 'MultiKeySlice'. They are [EntryPoint('MultiKeySlice', 'splash_ingest.handlers', 'MultiKeyHDF5DatasetSliceHandler', None), EntryPoint('MultiKeySlice', 'splash_ingest.handlers', 'MultiKeyHDF5DatasetSliceHandler', None), EntryPoint('MultiKeySlice', 'splash_ingest.handlers', 'MultiKeyHDF5DatasetSliceHandler', None)]. The match EntryPoint('MultiKeySlice', 'splash_ingest.handlers', 'MultiKeyHDF5DatasetSliceHandler', None) has won the race.
  warnings.warn(


In [6]:

from databroker.projector import project_xarray

xarray = project_xarray(run)
xarray

<xarray.Dataset>
Dimensions:  (dim_0: 16, dim_1: 9, time: 4)
Coordinates:
  * time     (time) float64 1.602e+09 1.602e+09 1.602e+09 1.602e+09
Dimensions without coordinates: dim_0, dim_1
Data variables:
    data     (time, dim_0, dim_1) float64 dask.array<chunksize=(1, 16, 9), meta=np.ndarray>
    darks    (time, dim_0, dim_1) float64 dask.array<chunksize=(4, 16, 9), meta=np.ndarray>
Attributes:
    sample_name:      nifty sample
    instrument_name:  my station